<a href="https://colab.research.google.com/github/sominshim/Algorithm/blob/main/tistory_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting

In [2]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [3]:
import time, os
from selenium import webdriver
import requests
from bs4 import BeautifulSoup

from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert
import time
from tqdm.notebook import tqdm
from pandas import Series as Series
from pandas import DataFrame as DataFrame

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [13]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Crawler

In [101]:
raw_df

,날짜,네이버/티스토리,관광 명소,제목,url,글쓴이


In [116]:
# 데이터 프레임 생성
raw_df = DataFrame(columns=['날짜', '네이버/티스토리', '관광 명소', '제목', '내용', 'url', '글쓴이'])

# 드라이버 지정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

# 페이지 수 초기화
page = 1
limit_page = 31 # 마지막 페이지.. 수동 확인
article_cnt = 1

keyword = "천지연폭포"

while page <= 31:
    while (page <= 31):
        url = "https://search.daum.net/search?w=blog&f=section&SA=tistory&lpp=10&nil_src=tistory&q={}&period=u&sd=20200301000000&ed=20211001235959&p={}".format(keyword, page)
        driver.get(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # pages = len(driver.find_elements_by_class_name("link_page"))
        pages = 57

        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector(".type_fulltext .list_info li"))

        # 한 페이지 안의 기사 수 만큼 탐색
        for i in range(1, t_len+1):
            if t_len == 1:
                a = driver.find_element_by_css_selector(".type_fulltext li .wrap_tit .f_link_b")
                b = driver.find_element_by_css_selector(".type_fulltext .wrap_cont .info")
                c = driver.find_element_by_css_selector(".type_fulltext .wrap_cont .date")

            else:
                a = driver.find_element_by_css_selector(".type_fulltext li:nth-child({}) .wrap_tit .f_link_b".format(i))
                b = driver.find_element_by_css_selector(".type_fulltext li:nth-child({}) .wrap_cont .info".format(i))
                c = driver.find_element_by_css_selector(".type_fulltext li:nth-child({}) .wrap_cont .date".format(i))

            # 제목 수집
            tit = a.text
            
            # url, writer
            txt = b.text
            url_writer = txt.split('|')[0]
            url = 'https://'+ url_writer.split()[0]
            writer = "".join(url_writer.split()[1:])

            # date
            date = c.text

            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)

            # 본문
            content_list = []
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            tags = soup.select('div.tt_article_useless_p_margin p')
            for tag in tags:
                content = tag.text
                content = content.replace('\xa0', '')
                content_list.append(content)
                
            contents = ''.join(content_list)

            # 데이터 프레임에 삽입
            # 날짜, 네이버/티스토리, 관광 명소, 제목, 내용, url, 글쓴이
            raw_df.loc[article_cnt] = [date, '티스토리', keyword, tit, contents, url, writer]
            article_cnt += 1
            
            print(date)
            print(tit)
            print("-" * 20)

        page += 1
            
driver.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


2021.09.01
[서귀포시_천지동] 서귀포 대표 관광지, '천지연 폭포'
--------------------
2021.09.23
관광 하루 코스 - 서귀포편(천지연폭포, 외돌개, 황우지해안, 매일올레...
--------------------
2021.09.23
제주 서귀포- 천지연전복해물뚝배기(천지연폭포)
--------------------
2021.09.02
[제주도 한 달 살기] 4일차.흑염소목장,천지연폭포,탐라승마장
--------------------


KeyboardInterrupt: ignored

In [87]:
save_dir = '/content/drive/MyDrive/TR_코로나 데이터분석/data'

In [77]:
len(total_content)

303

In [118]:
raw_df.head()

,날짜,네이버/티스토리,관광 명소,제목,내용,url,글쓴이
1,2021.09.01,티스토리,천지연폭포,"[서귀포시_천지동] 서귀포 대표 관광지, '천지연 폭포'",안녕하세요!!오늘 소개해 드릴 곳은서귀포의 대표 관광지인'천지연 폭포' 입니다~!!...,https://raonlife.tistory.com/64,RaonLife
2,2021.09.23,티스토리,천지연폭포,"관광 하루 코스 - 서귀포편(천지연폭포, 외돌개, 황우지해안, 매일올레...",안녕하세요. 오리너구리 입니다.실제로 저희 부부가 하루만에 다녀온 서귀포&중문 1일...,https://lethe0115.tistory.com/45,오리너구리의일상기...
3,2021.09.23,티스토리,천지연폭포,제주 서귀포- 천지연전복해물뚝배기(천지연폭포),전기차 충전도 하고 관광도 할 겸 서귀포 천지연폭포에 가보기로 했습니다.근데 예상치...,https://yellowscooter.tistory.com/13,희스토리_A블로그
4,2021.09.02,티스토리,천지연폭포,"[제주도 한 달 살기] 4일차.흑염소목장,천지연폭포,탐라승마장",,https://takemenow.tistory.com/346,핑계가사라지면꿈★...


In [89]:
raw_df.to_csv(save_dir+'/{}_data.csv'.format(keyword), index=False)